01_OneDrive_Ubicaciones_Clientes

In [1]:
import re
import numpy as np
import pandas as pd
import requests
from time import sleep
from typing import Tuple, Optional, Dict
import warnings
warnings.filterwarnings('ignore')


In [2]:
# ============================================================================
# CONFIGURACIÓN GLOBAL - EJECUTAR ESTA CELDA PRIMERO
# ============================================================================

import requests
from time import sleep
from typing import Dict, Tuple, Optional

# Límites geográficos de Uruguay para validación
URUGUAY_BOUNDS = {
    'lat_min': -35.0,
    'lat_max': -30.0,
    'lon_min': -58.5,
    'lon_max': -53.0
}

# Configuración Nominatim (OpenStreetMap - GRATUITO)
NOMINATIM_CONFIG = {
    'base_url': 'https://nominatim.openstreetmap.org/search',
    'user_agent': 'DimalApp/1.0 (tucorreo@ejemplo.com)',  # ← CAMBIAR
    'timeout': 10,
    'delay': 1.5
}

print("✓ Configuración cargada correctamente")

✓ Configuración cargada correctamente


In [8]:
%cd C:\Users\Administrador.PUESTO09\Desktop\Dimal_app_ clientes\data\Sucia

C:\Users\Administrador.PUESTO09\Desktop\Dimal_app_ clientes\data\Sucia


In [9]:
df_ubicaciones = pd.read_excel("Ubicaciones.xlsx")

In [10]:
# 1. Leer el archivo Excel

#df_ubicaciones = pd.read_excel("Ubicaciones.xlsx", sheet_name='Ubicacion', engine='openpyxl')
#print("Datos leídos con éxito. Muestra de datos antes de la limpieza:")
df_ubicaciones.info()
df_ubicaciones.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3127 entries, 0 to 3126
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   CODIGO     748 non-null    float64       
 1   CODIGO2    747 non-null    object        
 2   UBICACIÓN  748 non-null    object        
 3   FECHA      748 non-null    datetime64[ns]
 4   ORIGEN     748 non-null    object        
 5   CODCLI2    3127 non-null   int64         
 6   TAREAS     2852 non-null   float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(3)
memory usage: 171.1+ KB


,CODIGO,CODIGO2,UBICACIÓN,FECHA,ORIGEN,CODCLI2,TAREAS
0,82122.0,NUEVOS RUMBOS SALINAS,"http://maps.google.com/?q=-34.7656659,-55.8378493",2025-07-26,agregados_adrian,82122,82122.0
1,81227.0,LA ESQUINA jacqueline vanesca (cobranza),"http://maps.google.com/?q=-34.78064530920805,-...",2025-07-26,agregados_adrian,81125,81125.0


In [11]:
# Dejo solo los que tienen ubicaciones.

df_ubicaciones = df_ubicaciones.dropna(subset=['UBICACIÓN'])

In [12]:
# 1. Leer el archivo Excel

#df_ubicaciones = pd.read_excel("Ubicaciones.xlsx", sheet_name='Ubicacion', engine='openpyxl')
#print("Datos leídos con éxito. Muestra de datos antes de la limpieza:")
df_ubicaciones.info()
df_ubicaciones.head(2)

<class 'pandas.core.frame.DataFrame'>
Index: 748 entries, 0 to 748
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   CODIGO     747 non-null    float64       
 1   CODIGO2    746 non-null    object        
 2   UBICACIÓN  748 non-null    object        
 3   FECHA      747 non-null    datetime64[ns]
 4   ORIGEN     747 non-null    object        
 5   CODCLI2    748 non-null    int64         
 6   TAREAS     474 non-null    float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(3)
memory usage: 46.8+ KB


,CODIGO,CODIGO2,UBICACIÓN,FECHA,ORIGEN,CODCLI2,TAREAS
0,82122.0,NUEVOS RUMBOS SALINAS,"http://maps.google.com/?q=-34.7656659,-55.8378493",2025-07-26,agregados_adrian,82122,82122.0
1,81227.0,LA ESQUINA jacqueline vanesca (cobranza),"http://maps.google.com/?q=-34.78064530920805,-...",2025-07-26,agregados_adrian,81125,81125.0


In [13]:
# Dejo solo los que tienen ubicaciones.

df_ubicaciones = df_ubicaciones.dropna(subset=['UBICACIÓN'])

In [14]:
df_ubicaciones.info()

<class 'pandas.core.frame.DataFrame'>
Index: 748 entries, 0 to 748
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   CODIGO     747 non-null    float64       
 1   CODIGO2    746 non-null    object        
 2   UBICACIÓN  748 non-null    object        
 3   FECHA      747 non-null    datetime64[ns]
 4   ORIGEN     747 non-null    object        
 5   CODCLI2    748 non-null    int64         
 6   TAREAS     474 non-null    float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(3)
memory usage: 46.8+ KB


BLOQUE 2: PROCESAMIENTO DE NOMBRES Y CÓDIGOS¶


Observaciones df_ubicaciones

La columna CODIGO2, incluye nombres, vamos a dejarlo con el nombre NOMCLI.
La columna** UBICACION** contiene los enlaces con coordenadas a Google Map podriamos proceder creando una columna Latitud y otra Longitud.
Procedemos a ejecutar lo descrito:

In [15]:
df_ubicaciones.rename(columns={'CODIGO2': 'NOMCLI'}, inplace=True)

FUNCIONES DE VALIDACIÓN (NUEVO)

In [16]:
def validate_coordinates(lat: float, lon: float) -> bool:
    """
    Valida que las coordenadas estén dentro de Uruguay.
    
    Args:
        lat: Latitud
        lon: Longitud
        
    Returns:
        True si las coordenadas están en Uruguay
    """
    return (URUGUAY_BOUNDS['lat_min'] <= lat <= URUGUAY_BOUNDS['lat_max'] and
            URUGUAY_BOUNDS['lon_min'] <= lon <= URUGUAY_BOUNDS['lon_max'])


FUNCIONES DE GEOCODING (NUEVO)

In [30]:
def resolve_short_url(url: str) -> Optional[str]:
    """
    Resuelve URLs acortadas (goo.gl) siguiendo redirecciones.
    
    Args:
        url: URL acortada
        
    Returns:
        URL completa o None si falla
    """
    try:
        response = requests.head(url, allow_redirects=True, timeout=5)
        return response.url
    except Exception as e:
        print(f"   ⚠ Error resolviendo URL: {e}")
        return None


def geocode_with_nominatim(ubicacion: str, nombre_cliente: str = "") -> Dict:
    """
    Geocodifica usando Nominatim (OpenStreetMap - GRATIS).
    
    Args:
        ubicacion: URL o dirección
        nombre_cliente: Nombre del cliente para contexto
        
    Returns:
        dict con 'LATITUD', 'LONGITUD', 'METODO', 'CONFIANZA'
    """
    # Paso 1: Si es URL acortada, intentar resolver
    if 'goo.gl' in ubicacion or 'maps.app.goo.gl' in ubicacion:
        print(f"   → Resolviendo URL acortada...")
        ubicacion_resuelta = resolve_short_url(ubicacion)
        
        if ubicacion_resuelta:
            # Intentar extraer coordenadas de la URL resuelta
            coords = extract_coordinates(ubicacion_resuelta)
            if coords['LATITUD'] is not None:
                coords['METODO'] = 'url_resuelta'
                coords['CONFIANZA'] = 80
                print(f"   ✓ Coordenadas extraídas de URL resuelta")
                return coords
    
    # Paso 2: Geocoding por nombre
    query = f"{nombre_cliente}, Uruguay" if nombre_cliente else "Uruguay"
    
    try:
        sleep(NOMINATIM_CONFIG['delay'])  # Respetar rate limit
        
        params = {
            'q': query,
            'format': 'json',
            'limit': 1,
            'countrycodes': 'uy'
        }
        
        headers = {
            'User-Agent': NOMINATIM_CONFIG['user_agent']
        }
        
        response = requests.get(
            NOMINATIM_CONFIG['base_url'],
            params=params,
            headers=headers,
            timeout=NOMINATIM_CONFIG['timeout']
        )
        
        if response.status_code == 200:
            results = response.json()
            if results:
                result = results[0]
                lat = float(result['lat'])
                lon = float(result['lon'])
                
                if validate_coordinates(lat, lon):
                    print(f"   ✓ Geocoding exitoso: {result.get('display_name', '')[:50]}...")
                    return {
                        'LATITUD': lat,
                        'LONGITUD': lon,
                        'METODO': 'nominatim_geocoding',
                        'CONFIANZA': 60
                    }
                else:
                    print(f"   ⚠ Coordenadas fuera de Uruguay")
        else:
            print(f"   ⚠ Error HTTP {response.status_code}")
            
    except Exception as e:
        print(f"   ⚠ Error en geocoding: {e}")
    
    return {
        'LATITUD': None,
        'LONGITUD': None,
        'METODO': 'geocoding_fallido',
        'CONFIANZA': 0
    }



CELDA 8: EXTRACCIÓN DE COORDENADAS (REEMPLAZA TU extract_coordinates)

In [31]:
def extract_coordinates(ubicacion) -> pd.Series:
    """
    Extrae coordenadas de URLs con múltiples patrones.
    Ahora retorna también METODO y CONFIANZA.
    """
    if pd.isna(ubicacion):
        return pd.Series({
            'LATITUD': None, 
            'LONGITUD': None, 
            'METODO': 'ubicacion_nula',
            'CONFIANZA': 0
        })

    ubicacion = str(ubicacion)
    
    # Patrones ordenados por confianza
    patterns = [
        (r'lat/lng:\s*\(\s*(-?\d+\.?\d*),\s*(-?\d+\.?\d*)\s*\)', 'lat_lng_parentesis', 95),
        (r'lat/lng:\s*(-?\d+\.?\d*),\s*(-?\d+\.?\d*)', 'lat_lng_directo', 95),
        (r'http[s]?://maps\.google\.com/\?q\s*=\s*(-?\d+\.?\d*)\s*,\s*(-?\d+\.?\d*)', 'google_q_param', 90),
        (r'@(-?\d+\.?\d*),\s*(-?\d+\.?\d*)', 'google_at', 90),
        (r'!3d\s*(-?\d+\.?\d*)!4d\s*(-?\d+\.?\d*)', 'google_3d4d', 85),
        (r'q=(-?\d+\.?\d*),\s*(-?\d+\.?\d*)', 'q_param', 85),
    ]

    for pattern, metodo, confianza in patterns:
        match = re.search(pattern, ubicacion)
        if match:
            try:
                lat = float(match.group(1))
                lon = float(match.group(2))
                
                # Validar coordenadas
                if validate_coordinates(lat, lon):
                    return pd.Series({
                        'LATITUD': lat, 
                        'LONGITUD': lon, 
                        'METODO': metodo,
                        'CONFIANZA': confianza
                    })
            except (ValueError, IndexError):
                continue

    # Búsqueda genérica de pares de números
    all_coords = re.findall(r'(-?\d+\.\d+)', ubicacion)
    if len(all_coords) >= 2:
        try:
            lat = float(all_coords[0])
            lon = float(all_coords[1])
            if validate_coordinates(lat, lon):
                return pd.Series({
                    'LATITUD': lat, 
                    'LONGITUD': lon, 
                    'METODO': 'coordenadas_genericas',
                    'CONFIANZA': 70
                })
        except ValueError:
            pass

    # No se encontraron coordenadas - marcar para geocoding
    return pd.Series({
        'LATITUD': None, 
        'LONGITUD': None, 
        'METODO': 'requiere_geocoding',
        'CONFIANZA': 0
    })


CELDA 9: PROCESAMIENTO PRINCIPAL (REEMPLAZA TU CELDA DE EXTRACCIÓN)

In [32]:
print("="*70)
print("PROCESAMIENTO DE UBICACIONES CON GEOCODING AUTOMÁTICO")
print("="*70)

# Paso 1: Extraer coordenadas con regex
print("\n[1/4] Extrayendo coordenadas de URLs...")
coords_data = df_ubicaciones['UBICACIÓN'].apply(extract_coordinates)

df_ubicaciones['LATITUD'] = coords_data['LATITUD']
df_ubicaciones['LONGITUD'] = coords_data['LONGITUD']
df_ubicaciones['METODO'] = coords_data['METODO']
df_ubicaciones['CONFIANZA'] = coords_data['CONFIANZA']

coords_extraidas = df_ubicaciones['LATITUD'].notna().sum()
print(f"✓ {coords_extraidas} coordenadas extraídas exitosamente")

# Paso 2: Identificar registros que necesitan geocoding
print("\n[2/4] Identificando registros para geocoding...")
mask_requiere_geocoding = (
    (df_ubicaciones['LATITUD'].isna()) & 
    (df_ubicaciones['UBICACIÓN'].notna())
)
registros_geocoding = df_ubicaciones[mask_requiere_geocoding]

print(f"→ {len(registros_geocoding)} registros requieren geocoding")

# Paso 3: Aplicar geocoding
if len(registros_geocoding) > 0:
    print("\n[3/4] Aplicando geocoding automático...")
    print("⏳ Esto puede tardar unos minutos (respetando rate limits)...")
    
    geocoding_exitoso = 0
    geocoding_fallido = 0
    
    for idx, row in registros_geocoding.iterrows():
        print(f"\n   Procesando {idx+1}/{len(registros_geocoding)}: {row.get('NOMCLI', 'Sin nombre')[:30]}...")
        
        result = geocode_with_nominatim(
            row['UBICACIÓN'], 
            str(row.get('NOMCLI', ''))
        )
        
        df_ubicaciones.at[idx, 'LATITUD'] = result['LATITUD']
        df_ubicaciones.at[idx, 'LONGITUD'] = result['LONGITUD']
        df_ubicaciones.at[idx, 'METODO'] = result['METODO']
        df_ubicaciones.at[idx, 'CONFIANZA'] = result['CONFIANZA']
        
        if result['LATITUD'] is not None:
            geocoding_exitoso += 1
        else:
            geocoding_fallido += 1
    
    print(f"\n✓ Geocoding completado:")
    print(f"  - Exitosos: {geocoding_exitoso}")
    print(f"  - Fallidos: {geocoding_fallido}")
else:
    print("\n[3/4] No hay registros que requieran geocoding")

# Paso 4: Validación final
print("\n[4/4] Validando coordenadas finales...")
mask_coords = df_ubicaciones['LATITUD'].notna() & df_ubicaciones['LONGITUD'].notna()
coords_invalidas = df_ubicaciones[mask_coords].apply(
    lambda row: not validate_coordinates(row['LATITUD'], row['LONGITUD']),
    axis=1
)

if coords_invalidas.any():
    print(f"⚠ {coords_invalidas.sum()} coordenadas fuera de Uruguay detectadas")
    df_ubicaciones.loc[coords_invalidas[coords_invalidas].index, 'METODO'] = 'coordenadas_invalidas'
    df_ubicaciones.loc[coords_invalidas[coords_invalidas].index, 'CONFIANZA'] = 0

# Exportar casos problemáticos actualizados
filas_problematicas = df_ubicaciones[
    (df_ubicaciones['LATITUD'].isna()) | 
    (df_ubicaciones['CONFIANZA'] < 60)
]
filas_problematicas.to_excel('ubicaciones_revision_manual.xlsx', index=False)
print(f"\n→ {len(filas_problematicas)} registros requieren revisión manual")
print("  Exportados a: ubicaciones_revision_manual.xlsx")


PROCESAMIENTO DE UBICACIONES CON GEOCODING AUTOMÁTICO

[1/4] Extrayendo coordenadas de URLs...
✓ 740 coordenadas extraídas exitosamente

[2/4] Identificando registros para geocoding...
→ 8 registros requieren geocoding

[3/4] Aplicando geocoding automático...
⏳ Esto puede tardar unos minutos (respetando rate limits)...

   Procesando 8/8: 80319,ALMACEN QUIBOW...
   → Resolviendo URL acortada...
   ✓ Coordenadas extraídas de URL resuelta

   Procesando 9/8: 62619, LETICIA COSTA...
   → Resolviendo URL acortada...
   ✓ Coordenadas extraídas de URL resuelta

   Procesando 164/8: CASA LAUTARO...
   → Resolviendo URL acortada...
   ✓ Coordenadas extraídas de URL resuelta

   Procesando 197/8: EMPRESA...
   → Resolviendo URL acortada...
   ✓ Coordenadas extraídas de URL resuelta

   Procesando 198/8: CASA DARIO...
   → Resolviendo URL acortada...
   ✓ Coordenadas extraídas de URL resuelta

   Procesando 200/8: CASA CONRADO...
   → Resolviendo URL acortada...
   ✓ Coordenadas extraídas de URL

CELDA 10: GENERAR REPORTES (NUEVO)

In [33]:
def generate_validation_report(df: pd.DataFrame) -> pd.DataFrame:
    """Genera reporte detallado de validación."""
    
    total = len(df)
    con_coords = df['LATITUD'].notna().sum()
    sin_coords = total - con_coords
    revision_manual = (
        (df['LATITUD'].isna()) | 
        (df['CONFIANZA'] < 60)
    ).sum()
    
    report_data = {
        'Métrica': [
            'Total registros procesados',
            'Con coordenadas exitosas',
            'Sin coordenadas',
            'Tasa de éxito',
            '---',
            'Requieren revisión manual',
            'Listos para uso',
            '---'
        ],
        'Valor': [
            total,
            con_coords,
            sin_coords,
            f"{con_coords/total*100:.1f}%" if total > 0 else "0%",
            '',
            revision_manual,
            total - revision_manual,
            ''
        ]
    }
    
    # Agregar desglose por método
    report_data['Métrica'].append('=== Métodos de obtención ===')
    report_data['Valor'].append('')
    
    for metodo, cantidad in df['METODO'].value_counts().items():
        report_data['Métrica'].append(f"  {metodo}")
        report_data['Valor'].append(f"{cantidad} ({cantidad/total*100:.1f}%)")
    
    return pd.DataFrame(report_data)

print("\n" + "="*70)
print("REPORTE DE VALIDACIÓN")
print("="*70)
report_df = generate_validation_report(df_ubicaciones)
print(report_df.to_string(index=False))


REPORTE DE VALIDACIÓN
                     Métrica       Valor
  Total registros procesados         748
    Con coordenadas exitosas         748
             Sin coordenadas           0
               Tasa de éxito      100.0%
                         ---            
   Requieren revisión manual           0
             Listos para uso         748
                         ---            
=== Métodos de obtención ===            
              google_q_param 615 (82.2%)
          lat_lng_parentesis  80 (10.7%)
                   google_at   27 (3.6%)
             lat_lng_directo   18 (2.4%)
                url_resuelta    8 (1.1%)


In [40]:
# Eliminar datos deempresa que ya estan apartados. 

codigos_eliminar = [100001, 100002, 100003, 100004, 100000, 100005]

df_ubicaciones = df_ubicaciones[~df_ubicaciones["CODIGO"].isin(codigos_eliminar)]


CELDA 11: LIMPIEZA FINAL Y GUARDADO


In [43]:
print("\n" + "="*70)
print("GUARDANDO RESULTADOS")
print("="*70)

# Crear DataFrame final limpio
ubi_cliente = df_ubicaciones[
    ['CODIGO', 'NOMCLI', 'LATITUD', 'LONGITUD', 'UBICACIÓN', 'FECHA', 'METODO', 'CONFIANZA']
].copy()

# Conversiones de tipos
ubi_cliente['LONGITUD'] = pd.to_numeric(ubi_cliente['LONGITUD'], errors='coerce')
ubi_cliente['LATITUD'] = pd.to_numeric(ubi_cliente['LATITUD'], errors='coerce')
ubi_cliente['CODIGO'] = pd.to_numeric(ubi_cliente['CODIGO'], errors='coerce')
ubi_cliente['FECHA'] = pd.to_datetime(ubi_cliente['FECHA'], errors='coerce')
ubi_cliente['NOMCLI'] = ubi_cliente['NOMCLI'].astype(str)

# Separar registros limpios vs revisión manual
ubi_cliente_limpio = ubi_cliente[
    (ubi_cliente['LATITUD'].notna()) & 
    (ubi_cliente['CONFIANZA'] >= 60)
].copy()

ubi_revision_manual = ubi_cliente[
    (ubi_cliente['LATITUD'].isna()) | 
    (ubi_cliente['CONFIANZA'] < 60)
].copy()

# Eliminar duplicados en datos limpios
filas_antes = len(ubi_cliente_limpio)
ubi_cliente_limpio = ubi_cliente_limpio.drop_duplicates(subset=['NOMCLI'])
ubi_cliente_limpio.reset_index(drop=True, inplace=True)
print(f"→ Eliminados {filas_antes - len(ubi_cliente_limpio)} duplicados")

# Guardar resultados en Excel con múltiples hojas
try:
    ruta_salida = "../Limpia/Ubicaciones_direcciones.xlsx"
    
    with pd.ExcelWriter(ruta_salida, engine='openpyxl') as writer:
        # Hoja 1: Datos limpios listos para usar
        ubi_cliente_limpio.to_excel(
            writer, 
            sheet_name='Ubicaciones_Limpias', 
            index=False
        )
        
        # Hoja 2: Requieren revisión manual
        ubi_revision_manual.to_excel(
            writer, 
            sheet_name='Revision_Manual', 
            index=False
        )
        
        # Hoja 3: Reporte de validación
        report_df.to_excel(
            writer, 
            sheet_name='Reporte_Validacion', 
            index=False
        )
        
        # Hoja 4: Metadata
        metadata = pd.DataFrame([
            ['Fecha de procesamiento', pd.Timestamp.now()],
            ['Total registros originales', len(df_ubicaciones)],
            ['Registros limpios finales', len(ubi_cliente_limpio)],
            ['Registros para revisión', len(ubi_revision_manual)],
            ['Tasa de éxito final', f"{len(ubi_cliente_limpio)/len(df_ubicaciones)*100:.1f}%"]
        ], columns=['Campo', 'Valor'])
        
        metadata.to_excel(writer, sheet_name='Metadata', index=False)
    
    print(f"\n✓ Archivo guardado exitosamente: {ruta_salida}")
    print(f"\nContenido del archivo:")
    print(f"  📄 Hoja 'Ubicaciones_Limpias': {len(ubi_cliente_limpio)} registros")
    print(f"  📄 Hoja 'Revision_Manual': {len(ubi_revision_manual)} registros")
    print(f"  📄 Hoja 'Reporte_Validacion': Estadísticas detalladas")
    print(f"  📄 Hoja 'Metadata': Información del procesamiento")
    
except Exception as e:
    print(f"❌ Error al guardar: {e}")
    raise

print("\n" + "="*70)
print("✓ PROCESO COMPLETADO EXITOSAMENTE")
print("="*70)


GUARDANDO RESULTADOS
→ Eliminados 5 duplicados

✓ Archivo guardado exitosamente: ../Limpia/Ubicaciones_direcciones.xlsx

Contenido del archivo:
  📄 Hoja 'Ubicaciones_Limpias': 738 registros
  📄 Hoja 'Revision_Manual': 0 registros
  📄 Hoja 'Reporte_Validacion': Estadísticas detalladas
  📄 Hoja 'Metadata': Información del procesamiento

✓ PROCESO COMPLETADO EXITOSAMENTE


CELDA 12: VERIFICACIÓN FINAL (OPCIONAL)

In [42]:
print("\n📊 RESUMEN FINAL:")
print(f"  - Registros procesados: {len(df_ubicaciones)}")
print(f"  - Con coordenadas válidas: {len(ubi_cliente_limpio)} ({len(ubi_cliente_limpio)/len(df_ubicaciones)*100:.1f}%)")
print(f"  - Requieren revisión: {len(ubi_revision_manual)} ({len(ubi_revision_manual)/len(df_ubicaciones)*100:.1f}%)")

print("\n🔍 MÉTODOS UTILIZADOS:")
for metodo, count in ubi_cliente['METODO'].value_counts().head(5).items():
    print(f"  - {metodo}: {count}")

print("\n📍 MUESTRAS DE COORDENADAS:")
print(ubi_cliente_limpio[['NOMCLI', 'LATITUD', 'LONGITUD', 'CONFIANZA']].head(3))


📊 RESUMEN FINAL:
  - Registros procesados: 743
  - Con coordenadas válidas: 738 (99.3%)
  - Requieren revisión: 0 (0.0%)

🔍 MÉTODOS UTILIZADOS:
  - google_q_param: 615
  - lat_lng_parentesis: 80
  - google_at: 27
  - lat_lng_directo: 18
  - url_resuelta: 3

📍 MUESTRAS DE COORDENADAS:
                                     NOMCLI    LATITUD   LONGITUD  CONFIANZA
0                     NUEVOS RUMBOS SALINAS -34.765666 -55.837849         90
1  LA ESQUINA jacqueline vanesca (cobranza) -34.780645 -55.838592         90
2                        11941, LO DE RAMON -34.766682 -55.742948         90
